Подключаем библиотеки

In [ ]:
! pip install gspread

In [ ]:
! pip install df2gspread

In [ ]:
import pandas as pd

In [ ]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
import httplib2 
import apiclient.discovery

## Вариант 1

In [92]:
import gspread

from google.oauth2.service_account import Credentials
# доступы к Гугл диску и Гугл таблицам
scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

# авторизация, указываем файл с ключевой информацией (скачивается с 
# https://console.cloud.google.com/projectselector2/iam-admin/serviceaccounts?supportedpurview=project)
# выбираем проект, добавляем ключ Add Key
credentials = Credentials.from_service_account_file(
    'XXXXX.json',
    scopes=scopes
)

gc = gspread.authorize(credentials)
# открываем таблицу - указываем имя таблицы
sh = gc.open('pythontable')

# считываем таблицу
print(sh.sheet1.get_all_values())

[['name', 'value'], ['ann', '5'], ['kate', '10']]


## Вариант 2

In [97]:
scope = [ 'https://www.googleapis.com/auth/spreadsheets',
         'https://www.googleapis.com/auth/drive']

# Authorization
credentials = ServiceAccountCredentials.from_json_keyfile_name('XXX.json', scope)
gs = gspread.authorize(credentials)

In [98]:
# Name of the table in google sheets, 
# can be url for open_by_url
# or id (key) part for open_by_key
table_name = 'pythontable'  # Your table
# Get this table
work_sheet = gs.open(table_name)

In [99]:
# Select 1st sheet
sheet1 = work_sheet.sheet1

# Get data in python lists format
data = sheet1.get_all_values()

In [100]:
data

[['name', 'value'], ['ann', '5'], ['kate', '10']]

## Вариант 3

In [87]:
CREDENTIALS_FILE = 'XXX.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_FILE, ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive'])
httpAuth = credentials.authorize(httplib2.Http()) # Авторизуемся в системе
service = apiclient.discovery.build('sheets', 'v4', http = httpAuth) # Выбираем работу с таблицами и 4 версию API 

In [89]:
ranges = ["sheet1"] # 
spreadsheetId='1c9eQiBm6kcgW6JK_8A7UYUQPdWJ_lXRF-s9030f_VUY'
results = service.spreadsheets().values().batchGet(spreadsheetId = spreadsheetId, 
                                     ranges = ranges, 
                                     valueRenderOption = 'FORMATTED_VALUE',  
                                     dateTimeRenderOption = 'FORMATTED_STRING').execute() 
sheet_values = results['valueRanges'][0]['values']
print(sheet_values)

[['name', 'value'], ['ann', '5'], ['kate', '10']]


Работаем со считанными данными как с дата-фреймом

In [101]:
data

[['name', 'value'], ['ann', '5'], ['kate', '10']]

In [102]:
headers = data.pop(0)

In [103]:
headers

['name', 'value']

In [104]:
df = pd.DataFrame(data, columns=headers)

In [105]:
df

,name,value
0,ann,5
1,kate,10


In [107]:
df.sort_values('value', ascending=False)

,name,value
0,ann,5
1,kate,10


In [108]:
# Добавили строку
sheet1.append_row([ 'bob', 500])

{'spreadsheetId': '1c9eQiBm6kcgW6JK_8A7UYUQPdWJ_lXRF-s9030f_VUY',
 'tableRange': 'sheet1!A1:B3',
 'updates': {'spreadsheetId': '1c9eQiBm6kcgW6JK_8A7UYUQPdWJ_lXRF-s9030f_VUY',
  'updatedRange': 'sheet1!A4:B4',
  'updatedRows': 1,
  'updatedColumns': 2,
  'updatedCells': 2}}

In [110]:
sheet1.get_all_values()

[['name', 'value'], ['ann', '5'], ['kate', '10'], ['bob', '500']]

In [115]:
# Looks like spreadsheet should be already present at the dist (so, run code in create table section)
spreadsheet_name = 'A new spreadsheet'
sheet = 'new_sheat1'
d2g.upload(df, spreadsheet_name, sheet, credentials=credentials, row_names=True)

<Worksheet 'new_sheat1' id:241503458>